# Описание данных.

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов, в котором можно быстро узнать рыночную стоимость своего автомобиля. Нам были предоставлены следующие данные: технические характеристики, комплектации и цены автомобилей. С помощью них нам необходимо построить модель для определения стоимости машины.<br>Заказчику важны:<br>- качество предсказания;<br>- скорость предсказания;<br>- время обучения.<br>Чтобы усилить исследование, попробуем на деле несколько разных моделей. А в конце сравним характеристики моделей: время обучения, время предсказания и точность результата.

## Подготовка данных.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder,
                                   StandardScaler, 
                                   MinMaxScaler, 
                                   LabelEncoder, 
                                   LabelBinarizer)
from sklearn.impute import SimpleImputer

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
from sklearn.model_selection import cross_val_score

In [2]:
!pip install --upgrade scikit-learn

### Загрузка данных.

In [3]:
df_autos = pd.read_csv('/datasets/autos.csv')
df_autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df_autos.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


**Промежуточный вывод:**<br>На данном этапе мало, что можно сказать, однако стоит заметить, что придется поработать с названиями столбцов, с пропущенными значениями, а также в некоторых столбцах видны невозможные значения (аномалии).

### Работа с названиями столбцов.

In [6]:
df_autos.columns = ['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
                    'power', 'model', 'kilometer', 'registration_month', 'fuel_type', 'brand',
                    'repaired', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen']
df_autos.head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [7]:
df_autos = df_autos.drop(['number_of_pictures', 'postal_code', 'date_created', 'last_seen', 'registration_month', 'date_crawled'], axis=1)

**Промежуточный вывод:**<br>Мы привели названия столбцов к наиболее читаемому виду, что является правилом 'хорошего тона'. Также мы удалили столбцы, которые нам не понадобятся.

### Работа с пропусками.

In [8]:
df_autos.isna().sum()

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64

In [9]:
df_autos['vehicle_type'] = df_autos['vehicle_type'].fillna('other')
df_autos['gearbox'] = df_autos['gearbox'].fillna('unknown')
df_autos['model'] = df_autos['model'].fillna('unknown')
df_autos['fuel_type'] = df_autos['fuel_type'].fillna(df_autos['fuel_type'].mode().iloc[0])
df_autos['repaired'] = df_autos['repaired'].fillna('unknown')

In [10]:
df_autos.isna().sum()

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

**Промежуточный вывод:**<br>Мною было принято решение заменить все пропуски наиболее часто встречающимися значениями, так как если бы я удалил все строки с пропусками, то было бы потеряно большое количество данных.

### Работа с дубликатами и аномалиями.

In [11]:
df_autos.duplicated().sum()

47177

In [12]:
df_autos = df_autos.drop_duplicates().reset_index(drop=True)

In [13]:
df_autos.duplicated().sum()

0

In [14]:
df_autos.columns

Index(['price', 'vehicle_type', 'registration_year', 'gearbox', 'power',
       'model', 'kilometer', 'fuel_type', 'brand', 'repaired'],
      dtype='object')

In [15]:
df_autos = df_autos.loc[df_autos['price'] > 1000]

In [16]:
df_autos['registration_year'].unique()

array([2011, 2004, 2001, 2008, 2014, 2005, 2007, 2009, 2002, 2018, 2017,
       1981, 1995, 1991, 2016, 1984, 2006, 1998, 2012, 2010, 2000, 1992,
       1997, 2013, 2003, 1999, 1996, 2015, 1990, 1994, 1989, 1993, 1976,
       1983, 1973, 1969, 1971, 1987, 1982, 1985, 1988, 1980, 1986, 1965,
       1970, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977, 1961,
       1966, 1975, 1910, 1963, 1964, 1958, 1967, 1956, 3200, 1960, 1941,
       8888, 1500, 4100, 1962, 1929, 1957, 1940, 1949, 9999, 2019, 1937,
       1951, 1959, 1953, 1954, 1234, 2900, 6000, 5911, 5000, 4000, 1948,
       1000, 1952, 8500, 1932, 1950, 3700, 1942, 7000, 1935, 1933, 1936,
       1923, 1930, 1001, 1944, 2500, 1943, 1938, 1934, 1928, 5555, 9000,
       1600, 1800, 1039, 3000, 2800, 1931, 4500, 7800, 1947, 1927, 7100,
       1946])

In [17]:
df_autos = df_autos.loc[(df_autos['registration_year'] <= 2016) & (df_autos['registration_year'] >= 1900)]

In [18]:
df_autos['registration_year'].unique()

array([2011, 2004, 2001, 2008, 2014, 2005, 2007, 2009, 2002, 1981, 1995,
       1991, 2016, 1984, 2006, 1998, 2012, 2010, 2000, 1992, 1997, 2013,
       2003, 1999, 1996, 2015, 1990, 1994, 1989, 1993, 1976, 1983, 1973,
       1969, 1971, 1987, 1982, 1985, 1988, 1980, 1986, 1965, 1970, 1945,
       1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977, 1961, 1966, 1975,
       1910, 1963, 1964, 1958, 1967, 1956, 1960, 1941, 1962, 1929, 1957,
       1940, 1949, 1937, 1951, 1959, 1953, 1954, 1948, 1952, 1932, 1950,
       1942, 1935, 1933, 1936, 1923, 1930, 1944, 1943, 1938, 1934, 1928,
       1931, 1947, 1927, 1946])

In [19]:
df_autos['power'].unique()

array([  190,   163,    75,    69,   109,   125,   105,   140,   131,
         136,   102,   160,   231,   193,    99,    50,   113,   218,
         122,     0,   129,    70,   306,    95,    61,   177,   170,
          55,   143,   232,    60,   150,   156,    80,    82,   185,
          87,   180,    86,    84,   224,   101,   235,   200,   178,
         265,    77,   110,   120,   286,   116,    90,   184,   126,
         204,   194,    64,   305,   197,   179,   250,    88,    45,
         313,    41,    98,   115,   130,   211,   201,   213,    83,
         174,   100,   220,    54,    73,   192,    68,   299,    74,
          58,    52,   147,   310,    71,    97,    65,   239,   203,
           5,   300,    85,   258,   320,    63,    81,    44,   145,
         280,   260,   104,   333,   186,   117,   141,   132,   165,
         155,   234,   158,    92,    51,   135,    59,   107,   230,
         103,   209,   146,    67,   106,   166,   276,   344,    72,
         249,   237,

In [20]:
df_autos = df_autos.loc[df_autos['power'] <= 700]

In [21]:
df_autos['power'] = df_autos['power'].replace(0, df_autos['power'].median())

In [22]:
df_autos['power'].unique()

array([190., 163.,  75.,  69., 109., 125., 105., 140., 131., 136., 102.,
       160., 231., 193.,  99.,  50., 113., 218., 122., 116., 129.,  70.,
       306.,  95.,  61., 177., 170.,  55., 143., 232.,  60., 150., 156.,
        80.,  82., 185.,  87., 180.,  86.,  84., 224., 101., 235., 200.,
       178., 265.,  77., 110., 120., 286.,  90., 184., 126., 204., 194.,
        64., 305., 197., 179., 250.,  88.,  45., 313.,  41.,  98., 115.,
       130., 211., 201., 213.,  83., 174., 100., 220.,  54.,  73., 192.,
        68., 299.,  74.,  58.,  52., 147., 310.,  71.,  97.,  65., 239.,
       203.,   5., 300.,  85., 258., 320.,  63.,  81.,  44., 145., 280.,
       260., 104., 333., 186., 117., 141., 132., 165., 155., 234., 158.,
        92.,  51., 135.,  59., 107., 230., 103., 209., 146.,  67., 106.,
       166., 276., 344.,  72., 249., 237., 245., 111., 326., 279.,  23.,
       138., 175.,  96., 114., 226., 301., 334., 118., 128., 133., 124.,
       219., 241., 167.,  94., 182., 196., 238., 22

**Промежуточный вывод:**<br>Работа с явными и неявными дубликатами была проведена успешно. Также на данном этапе мы избавлись от аномалий, которые помешали бы нам на следующих этапах проекта.

**Общий вывод по шагу 1:**<br>При выполнении данного шага мы произвели предобработку данных. Сначала мы провели работу над названиями столбцов, затем проработали пропущенные значения, дубликаты и аномалии. Теперь мы имеем датасет, который готов к дальнейшей работе.

## Обучение и анализ моделей.

### Подготовка к обучению.

Чтобы автоматизировать процесс обучения, воспользуемся пайплайнами.

In [23]:
TEST_SIZE = 25
RANDOM_STATE = 42

In [24]:
X = df_autos.drop('price', axis = 1)
y = df_autos['price']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE
)

In [26]:
ohe_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired'] 
num_columns = ['registration_year', 'power', 'kilometer'] 

In [27]:
ohe_pipe = Pipeline([
    ('simpleImputer_ohe_one', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('simpleImputer_ohe_two', SimpleImputer(missing_values=' ', strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

In [28]:
data_preprocessor = ColumnTransformer([
    ('ohe', ohe_pipe, ohe_columns),
    ('num', MinMaxScaler(), num_columns)
], remainder='passthrough')

In [29]:
models_1 = {
    'LightGBM': lgb.LGBMRegressor(random_state=RANDOM_STATE)
}

models_2 = {
    'LinearRegression': LinearRegression()
}

In [30]:
pipe_final_1 = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', lgb.LGBMRegressor(random_state=RANDOM_STATE))
    ]
)

pipe_final_1.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('simpleImputer_ohe_one',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('simpleImputer_ohe_two',
                                                                   SimpleImputer(missing_values=' ',
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['vehicle_type', 'gearbox',
                                                   'model', 'fuel_type',
                                                   'brand', 'repaired']),
                                                 ('num', MinMaxScaler(),
                                                  ['registration_year', 'power',
                                                   'kilometer'])])),
                ('models', LGBMRegressor(random_state=42))])

In [31]:
pipe_final_2 = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models',  LinearRegression())
    ]
)

pipe_final_2.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('simpleImputer_ohe_one',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('simpleImputer_ohe_two',
                                                                   SimpleImputer(missing_values=' ',
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['vehicle_type', 'gearbox',
                                                   'model', 'fuel_type',
                                                   'brand', 'repaired']),
                                                 ('num', MinMaxScaler(),
                                                  ['registration_year', 'power',
                                                   'kilometer'])])),
                ('models', LinearRegression())])

### Обучение и анализ.

In [32]:
param_grid_1 = [
    {
        'models': [lgb.LGBMRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': [2, 3],
        'preprocessor__num': [StandardScaler(), 'passthrough']
    }
]

grid_search_1 = GridSearchCV(
    pipe_final_1,
    param_grid_1,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search_1.fit(X_train, y_train)

best_model = grid_search_1.best_estimator_

print('Лучшая модель для LGBM и её параметры:\n\n', best_model)

cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_value_lgbm = np.sqrt(-cv_scores.mean())
print('Метрика RMSE лучшей модели LGBM на тренировочной выборке:', rmse_value_lgbm)

start_time = time.time()
best_model.fit(X_train, y_train)
end_time = time.time()
train_time_lgbm = round((end_time - start_time), 5)
print("Время обучения LGBM: %s секунд" % train_time_lgbm)

start_time = time.time()
best_model.predict(X_train)
end_time = time.time()
predict_time_lgbm = round((end_time - start_time), 5)
print("Время предсказания LGBM: %s секунд" % predict_time_lgbm)

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Лучшая модель для LGBM и её параметры:

 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('simpleImputer_ohe_one',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('simpleImputer_ohe_two',
                                                                   SimpleImputer(missing_values=' ',
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                           

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Метрика RMSE лучшей модели LGBM на тренировочной выборке: 2055.21161251142
Время обучения LGBM: 3.60818 секунд
Время предсказания LGBM: 1.30042 секунд


In [33]:
param_grid_2 = [
    {
        'models': [LinearRegression()],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough'] 
    }
]

grid_search_2 = GridSearchCV(
    pipe_final_2,
    param_grid_2,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search_2.fit(X_train, y_train)

best_model_ = grid_search_2.best_estimator_

print('Лучшая модель для LR и её параметры:\n\n', best_model_)

cv_scores = cross_val_score(best_model_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_value_lr = np.sqrt(-cv_scores.mean())
print('Метрика RMSE лучшей модели LR на тренировочной выборке:', rmse_value_lr)

start_time = time.time()
best_model_.fit(X_train, y_train)
end_time = time.time()
train_time_lr = round((end_time - start_time), 5)
print("Время обучения LR: %s секунд" % train_time_lr)

start_time = time.time()
best_model_.predict(X_train)
end_time = time.time()
predict_time_lr = round((end_time - start_time), 5)
print("Время предсказания LR: %s секунд" % predict_time_lr)

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Лучшая модель для LR и её параметры:

 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('simpleImputer_ohe_one',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('simpleImputer_ohe_two',
                                                                   SimpleImputer(missing_values=' ',
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                             

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Метрика RMSE лучшей модели LR на тренировочной выборке: 2816.0284419681047
Время обучения LR: 35.72756 секунд
Время предсказания LR: 0.65871 секунд


In [34]:
results = pd.DataFrame({
    'Модель': ['LGBM', 'Linear Regression'],
    'RMSE на обучающей выборке': [rmse_value_lgbm, rmse_value_lr],
    'Время обучения': [train_time_lgbm, train_time_lr],
    'Время предсказания': [predict_time_lgbm, predict_time_lr]
})

results

,Модель,RMSE на обучающей выборке,Время обучения,Время предсказания
0,LGBM,2055.211613,3.60818,1.30042
1,Linear Regression,2816.028442,35.72756,0.65871


**Промежуточный вывод:**<br>Мы обучили две лучшие модели, а именно LightGBM и LinearRegression. В итоге, проанализировав время обучения, время предсказания и качество моделей на тестовой выборке, стоит сказать, что модель LGBM справляется лучше. Linear Regression выигрывает лишь по времени предсказанию, однако совершенно незначительно. Поэтому для данного проекта я выберу LightGBM.<br>Теперь выполним предсказания на тестовой выборке с помощью модели LGBM.

In [38]:
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Метрика RMSE лучшей модели на тестовой выборке:', round(rmse, 2))

Метрика RMSE лучшей модели на тестовой выборке: 2460.76


# Общий вывод.

В ходе данного проекта мы решали задачу определения рыночной стоимости подержанных автомобилей для сервиса по продаже автомобилей с пробегом «Не бит, не крашен». Наша цель заключалась в создании модели, способной быстро и точно определять стоимость автомобиля на основе его технических характеристик и комплектации.<br><br>Мы начали с предварительного анализа данных, который позволил нам выявить потенциальные проблемы, такие как пропущенные значения, аномалии и дубликаты. Затем мы провели предобработку данных, включая работу с названиями столбцов, заполнение пропущенных значений, удаление дубликатов и обработку аномалий.<br><br>На следующем этапе мы подготовили данные для обучения моделей, включая кодирование категориальных признаков и разделение данных на обучающий и тестовый наборы.<br><br>Затем мы приступили к обучению нескольких моделей, включая LightGBM и LinearRegression, с использованием метода поиска по сетке для настройки гиперпараметров. Мы анализировали время обучения, время предсказания и качество моделей на тестовой выборке.<br><br>В результате анализа мы пришли к выводу, что модель LightGBM лучше подходит для решения поставленной задачи. Она обладает более высокой точностью предсказаний и при этом имеет более низкое время обучения и предсказания по сравнению с моделью LinearRegression.<br><br>Таким образом, наша модель LightGBM может успешно использоваться сервисом для определения рыночной стоимости автомобилей и привлечения новых клиентов.